In [1]:
# TODO: Zhiyun


In [2]:
# 生产线每一台设备的状态

In [3]:
from imports import *

In [4]:
def get_equipment():
    
    df = pd.read_csv(os.path.join(BASE_DIR, 'equipment.csv'))
    return df

In [5]:
df = get_equipment()
df

,line,station,equipment_number,equipment_name
0,1,OP1,1,OP1-1
1,1,OP1,2,OP1-2
2,1,OP1,3,OP1-3
3,1,OP1,4,OP1-4
4,1,OP1,5,OP1-5
...,...,...,...,...
107,2,OP17,1,OP17-1
108,2,OP18,1,OP18-1
109,2,OP18,2,OP18-2
110,2,OP18,3,OP18-3


In [6]:
px.bar(df, x='station', facet_row='line')

In [6]:
# generate event per hour for each equipment, follow normal distribution with mean 1 and std 0.4
def generate_event_per_hour(df):
    df['event_per_hour'] = np.random.normal(1, 0.4, df.shape[0])
    df['event_name'] = 'Error'
    return df
df = generate_event_per_hour(df)

In [7]:
df

,line,station,equipment_number,equipment_name,event_per_hour,event_name
0,1,OP1,1,OP1-1,0.445968,Error
1,1,OP1,2,OP1-2,0.269812,Error
2,1,OP1,3,OP1-3,0.498566,Error
3,1,OP1,4,OP1-4,1.476657,Error
4,1,OP1,5,OP1-5,1.164823,Error
...,...,...,...,...,...,...
107,2,OP17,1,OP17-1,1.193263,Error
108,2,OP18,1,OP18-1,1.400451,Error
109,2,OP18,2,OP18-2,0.849182,Error
110,2,OP18,3,OP18-3,0.950393,Error


In [8]:
# df.to_csv(os.path.join(GE_DIR, 'equipment_event_summary.csv'), index=False)

In [12]:
# 生成符合分布的随机故障开始时间
# 开始，结束，状态，持续时间

# TODO: Zhiyun 时间改成全部的时间，从2018开始到2022，先不着急讨论后再完成


start = pd.to_datetime('2022-07-01')
end = pd.to_datetime('2022-07-31')

# 
status = 'Error'

# evemt_per_hour 代表每小时会发生event_per_hour次故障
event_per_hour = 1



# generate random datetime between start and end
def random_date(start, end, p):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = np.random.randint(int_delta)
    return start + timedelta(seconds=random_second)

# generate random event between start and end
def random_event(start, end, event_per_hour):

    
    delta_in_hours = (end - start).total_seconds() / 3600
    event_count = int(delta_in_hours * event_per_hour)
    # generate random datetime 
    event_start_list = [random_date(start, end, 1) for i in range(event_count)]
    # sort inplace
    event_start_list.sort()
    return event_start_list


df_event = pd.DataFrame()
df_event['start'] = random_event(start, end, event_per_hour)
df_event['status'] = status

In [14]:
df_event

,start,status
0,2022-07-01 00:22:48,Error
1,2022-07-01 01:08:36,Error
2,2022-07-01 03:20:56,Error
3,2022-07-01 03:48:10,Error
4,2022-07-01 04:44:45,Error
...,...,...
715,2022-07-30 15:47:51,Error
716,2022-07-30 20:37:59,Error
717,2022-07-30 22:16:24,Error
718,2022-07-30 22:22:33,Error


In [18]:
df

,line,station,equipment_number,equipment_name,event_per_hour,event_name
0,1,OP1,1,OP1-1,0.445968,Error
1,1,OP1,2,OP1-2,0.269812,Error
2,1,OP1,3,OP1-3,0.498566,Error
3,1,OP1,4,OP1-4,1.476657,Error
4,1,OP1,5,OP1-5,1.164823,Error
...,...,...,...,...,...,...
107,2,OP17,1,OP17-1,1.193263,Error
108,2,OP18,1,OP18-1,1.400451,Error
109,2,OP18,2,OP18-2,0.849182,Error
110,2,OP18,3,OP18-3,0.950393,Error


In [15]:
# iterate over each row in df and generate df_event for all rows

# 112row 相当于112台设备，每台设备都有一个event_per_hour，每小时有多少个故障

def generate_event(df, start, end):
    df_l = []
    for index, row in df.iterrows():
        df_event = pd.DataFrame()

        df_event['start'] = random_event(start, end, row['event_per_hour'])
        df_event['status'] = row['event_name']
        df_event['equipment_name'] = row['equipment_name']
        df_event['line'] = row['line']
        df_event['station'] = row['station']
        
        df_l.append(df_event)
    df_all = pd.concat(df_l)
        
    return df_all

In [16]:
df_event = generate_event(df, start, end)

In [17]:
df_event

,start,status,equipment_name,line,station
0,2022-07-01 04:45:11,Error,OP1-1,1,OP1
1,2022-07-01 08:07:06,Error,OP1-1,1,OP1
2,2022-07-01 10:03:32,Error,OP1-1,1,OP1
3,2022-07-01 15:31:19,Error,OP1-1,1,OP1
4,2022-07-01 16:28:23,Error,OP1-1,1,OP1
...,...,...,...,...,...
760,2022-07-30 17:40:31,Error,OP18-4,2,OP18
761,2022-07-30 19:57:04,Error,OP18-4,2,OP18
762,2022-07-30 20:31:50,Error,OP18-4,2,OP18
763,2022-07-30 20:35:52,Error,OP18-4,2,OP18


In [21]:
# 检查event的数量
df_event.groupby(['station','equipment_name']).count()

start  status  line
station equipment_name                     
OP1     OP1-1             778     778   778
        OP1-2             903     903   903
        OP1-3             851     851   851
        OP1-4            1735    1735  1735
        OP1-5            1211    1211  1211
        OP1-6            2003    2003  2003
OP10    OP10-1           1511    1511  1511
OP11    OP11-1           1840    1840  1840
OP12    OP12-1           1572    1572  1572
        OP12-10           909     909   909
        OP12-11          1496    1496  1496
        OP12-12          2171    2171  2171
        OP12-13          1310    1310  1310
        OP12-14          1240    1240  1240
        OP12-15          1938    1938  1938
        OP12-16           924     924   924
        OP12-17          1816    1816  1816
        OP12-18          1340    1340  1340
        OP12-19          1162    1162  1162
        OP12-2           1367    1367  1367
        OP12-20           798     798   798
        OP12-21          1673    1673  1673
        OP12-22          1890    1890  1890
        OP12-23          1806    1806  1806
        OP12-24           822     822   822
        OP12-3           1394    1394  1394
        OP12-4           1324    1324  1324
        OP12-5           2414    2414  2414
        OP12-6           1851    1851  1851
        OP12-7           1244    1244  1244
        OP12-8           2183    2183  2183
        OP12-9           1222    1222  1222
OP13    OP13-1           1994    1994  1994
        OP13-2           1426    1426  1426
        OP13-3           2023    2023  2023
        OP13-4           1688    1688  1688
        OP13-5           2133    2133  2133
OP14    OP14-6           1306    1306  1306
        OP14-7           1620    1620  1620
        OP14-8           1175    1175  1175
        OP14-9           1598    1598  1598
OP15    OP15-1           1062    1062  1062
OP16    OP16-1           1690    1690  1690
OP17    OP17-1           2141    2141  2141
OP18    OP18-1           2235    2235  2235
        OP18-2           1390    1390  1390
        OP18-3           1436    1436  1436
        OP18-4           1463    1463  1463
OP2     OP2-1            1035    1035  1035
OP3     OP3-1            1347    1347  1347
OP4     OP4-1            1649    1649  1649
OP5     OP5-1            1327    1327  1327
OP6     OP6-1             981     981   981
OP7     OP7-1            1606    1606  1606
OP8     OP8-1            1210    1210  1210
OP9     OP9-1            1580    1580  1580

In [22]:
# 生成截断正态分布的停线时长 单位分钟
from scipy.stats import truncnorm

def truncnorm_rv(mu, sigma, a, b):
    return truncnorm((a - mu) / sigma, (b - mu) / sigma, loc=mu, scale=sigma)

dist = truncnorm_rv(mu = 1.5, sigma = 2, a = 0, b = 100)

def random_duration():
    
    return int(dist.rvs(1)[0]*60)

In [24]:
# check distribution
# px.histogram(dist.rvs(1000))

In [25]:
df_event

,start,status,equipment_name,line,station
0,2022-07-01 04:45:11,Error,OP1-1,1,OP1
1,2022-07-01 08:07:06,Error,OP1-1,1,OP1
2,2022-07-01 10:03:32,Error,OP1-1,1,OP1
3,2022-07-01 15:31:19,Error,OP1-1,1,OP1
4,2022-07-01 16:28:23,Error,OP1-1,1,OP1
...,...,...,...,...,...
760,2022-07-30 17:40:31,Error,OP18-4,2,OP18
761,2022-07-30 19:57:04,Error,OP18-4,2,OP18
762,2022-07-30 20:31:50,Error,OP18-4,2,OP18
763,2022-07-30 20:35:52,Error,OP18-4,2,OP18


In [26]:
df_event['duration'] = df_event.apply(lambda x: random_duration(), axis=1)

In [27]:
df_event['end'] = df_event['start'] + pd.to_timedelta(df_event['duration'], unit='s')

In [18]:
df_event

,start,status,equipment_name,line,station,duration,end
0,2022-07-01 00:08:30,Error,OP1-1,1,OP1,32,2022-07-01 00:09:02
1,2022-07-01 01:47:28,Error,OP1-1,1,OP1,156,2022-07-01 01:50:04
2,2022-07-01 03:49:07,Error,OP1-1,1,OP1,9,2022-07-01 03:49:16
3,2022-07-01 04:54:00,Error,OP1-1,1,OP1,115,2022-07-01 04:55:55
4,2022-07-01 06:44:20,Error,OP1-1,1,OP1,146,2022-07-01 06:46:46
...,...,...,...,...,...,...,...
1046,2022-07-30 19:29:59,Error,OP18-4,2,OP18,131,2022-07-30 19:32:10
1047,2022-07-30 20:26:37,Error,OP18-4,2,OP18,17,2022-07-30 20:26:54
1048,2022-07-30 20:43:47,Error,OP18-4,2,OP18,105,2022-07-30 20:45:32
1049,2022-07-30 21:18:39,Error,OP18-4,2,OP18,195,2022-07-30 21:21:54


In [ ]:
# TODO：生成的event 会不会有overlapping的情况？ P3

# 输出Event Detail 此Notebook结束，没有完整的停线时间

In [19]:
df_event.to_csv(os.path.join(GE_DIR, 'equipment_event_detail.csv'), index=False)

In [37]:
# plot df_event with plotly showing start and end time with horizontal bars in the same line
def plot(df_event):
    
    fig = go.Figure()
    
    for station in df_event['station'].unique():
        df_station = df_event[df_event['station'] == station]
        for i, row in df_station.iterrows():
            
            show_legend = True if i == 0 else False
            
            fig.add_trace(go.Scatter(
                x=[row['start'], row['end']],
                y=[station,station],
                mode='lines',
                line=dict(color='red', width=10),
                name=row['status'],
                showlegend=show_legend
            ))
    # fig.update_yaxes(showticklabels=False)

    return fig

In [38]:
# plot for line 1 and station OP1 to OP8
fig = plot(df_event[(df_event['line'] == 1) & (df_event['station'].isin(['OP1', 'OP2', 'OP3', 'OP4', 'OP5', 'OP6', 'OP7', 'OP8']))])
fig